In [ ]:
import json, pathlib as pl, networkx as nx, pandas as pd, matplotlib.pyplot as plt
DATA = pl.Path("../data")

G     = nx.readwrite.json_graph.node_link_graph(
          json.load(open(DATA / "Tracks.json")))
stations = pd.read_json(DATA / "Stations.json")
yards    = pd.read_json(DATA / "Railyard-position.json")


In [ ]:
# Merge station attributes into graph nodes
attribs = stations.set_index("city").to_dict("index")
nx.set_node_attributes(G, attribs)


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
pos = nx.spring_layout(G, k=0.3, seed=42)

node_size = [50 + 8 * G.nodes[n]["n_tracks"] for n in G.nodes]
node_color = ["red" if n in yards["city"].values else "lightblue" for n in G.nodes]

nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color=node_color, ax=ax)
nx.draw_networkx_edges(G, pos, width=1.2, ax=ax)
nx.draw_networkx_labels(G, pos, font_size=8, ax=ax)
ax.set_title("Rail Network — Yard cities in red, node size = #tracks")
ax.axis("off")


In [ ]:
import folium
from folium.plugins import MarkerCluster

# centroid to center map
mean_lat = yards["lat"].mean()
mean_lon = yards["lon"].mean()
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=6)

# Draw track polylines
for _, row in pd.read_json(DATA / "Tracks.json").iterrows():
    latlngs = [(row["lat_a"], row["lon_a"]), (row["lat_b"], row["lon_b"])]
    folium.PolyLine(latlngs, weight=2).add_to(m)

# Station markers
cluster = MarkerCluster().add_to(m)
for _, row in stations.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=3 + 0.3 * row["n_tracks"],
        tooltip=f"{row['name']} ({row['n_tracks']} tracks)",
        color="blue",
        fill=True,
        fill_opacity=0.8
    ).add_to(cluster)

# Yard markers
for _, row in yards.iterrows():
    folium.Marker(
        location=[row["lat"], row["lon"]],
        icon=folium.Icon(color="red", icon="train", prefix="fa"),
        tooltip=f"Yard {row['yard_id']} — {row['city']}"
    ).add_to(m)

m  # Jupyter will render this
